In [2]:
#Required Functions

###################################################################
def braid_to_positive_braid(W):
    #Input is an array [1 -2 3 5 -1 -2] etc.
    #Output is a positive braid.  This could be adjusted to output just the positive array! 
    n=max(abs(i)+1 for i in W) #This is the braid group
    L=list(range(1,n)) #This will be used to generate the positive replacementwords
    positivewordreplacement=[]
    for j in range(1,n):
        positivewordreplacement.append(L[j-(n+1)::-1]+L+L[:j-1:-1]) #This array holds in each index the replacement positive word
        PositiveW=[];
    for i in W:
        if i>0:
            PositiveW.append(i)
        else:
             PositiveW.append(positivewordreplacement[i])
    PositiveW=flatten(PositiveW) #This is the positive braid word replacing the input W  
    Braidgroup=BraidGroup(n,'s'); #The generators are s_0-s_(n-2) I would like to change this to numbers but I dont know how yet.
    PositiveBraidW=Braidgroup(PositiveW)
    return PositiveBraidW
#############################################################################
def left_normal_to_string(leftnormbraid,n_from_braidgroup):
    #This will take a left normal braid, removes the garside elements and turns it back into a string 
    
   
    A=str(flatten(leftnormbraid))
    if A[1]=='1':                              #If there is not a garside element it puts a 1 there. This removes it. 
        A=A[4:-1]
    elif A.find('^') != -1:                    #This will remove the garside elements. We will have to duplicate the string however many times                                                 and append to the beginning.  For now it wont do that.
        A=A[A.find('^')+4:-1]
    else:
        A=A[2:-1]                             #This will remove the [ ] and just leave the braid word
    A=A.replace("*",',').replace(' ','')
    A=A[::-3]                                # This is me being sneaky, I grab only the numbers in reverse order, then flip back
    A=A[::-1]   
    print('This is the positive left normal form combed', A)                             # REMOVE THIS LATER!
    B=list(map(int, str(A)))                 #This is now in string format again.
    B = [x+1 for x in B]  #this changes it from 0-(n-2) to (1-n-1) like it should be! 

    return B
#####################################################################################

#This will partition the string into non-decreasing parts. The commented out part will also partition it into non-decreasing and also decreasing parts (there is a slight issue with parts of the braid word like 121 )

#def syllable_string_partitioning(B): 
#    syllable=[]
#    syllables=[]
#    for i in range(len(B)-1):
#        if (B[i]+1 == B[(i+1)] or B[i]-1==B[(i+1)]):
#            syllable.append(B[i])
#        else:
#            syllable.append(B[i])
#            syllables.append(syllable)
#            syllable=[]
#    return syllables
#################################################################################################
#If you want only increasing syllables you can run this one. 
def syllable_string_partitioning(B): 
    syllable=[]
    syllables=[]
    for i in range(len(B)-1):
        if (B[i]+1 == B[(i+1)]):
            syllable.append(B[i])
        else:
            syllable.append(B[i])
            syllables.append(syllable)
            syllable=[]
    return syllables
##############################################################################
def plat2rectdiag(syllablelist):
    # This is figuring out what braid group we're in by looking at the max crossing number. In a larger program this can be done sooner, and eliminated from this program.
    syllableends = [j for i, j in syllablelist]
    maxcrossing: int = max(syllableends)
    if maxcrossing % 2 == 1:
        n = (maxcrossing + 1) / 2
    else:
        n = (maxcrossing + 2) / 2
    n = int(n)

    # This is initializing the rectangular diagram: strands are initially in the first 2n positions, and each bridge starts on the next column.
    strands = list(range(1, (2 * n) + 1))
    vertexset = []
    i = 1
    while i <= n:
        vertexset.append((i, 2 * i - 1))
        vertexset.append((i, 2 * i))
        i += 1


    # For each syllable, we add two new vertices to the vertexset and update where each strand is
    # print(strands)
    k=1
    while syllablelist:
        # print('step ', k)
        currentsyllable = syllablelist[0]
        newvertextop = (n + k, strands[currentsyllable[0] - 1])
        newvertexbot = (n + k, strands[currentsyllable[1]] + 1)
        vsx = [i for i, j in vertexset]
        vsy = [j for i, j in vertexset]
        shiftedindices = list(filter(lambda x: vsy[x] >= strands[currentsyllable[1]] + 1, range(len(vsy))))
        for i in shiftedindices:
            vsy[i] = vsy[i] + 1
        vertexset = list(zip(vsx, vsy))
        vertexset = vertexset + [newvertextop, newvertexbot]
        # print(vertexset)

        # Now we need to update where the strands are. It's an annoying list manipulation, but nothing complicated.
        strands.insert(currentsyllable[1], strands[currentsyllable[1]])
        strandchange = list(range(currentsyllable[1] + 1, len(strands)))
        for i in strandchange:
            strands[i] += 1
        strands.pop(currentsyllable[0] - 1)
        # print(strands)
        syllablelist.pop(0)
        k += 1
    # print('end of for loop')
    # print('k = ',1)
    # print(strands)

    # Now we need to close off the plat where strands 1&2 are, 3&4 are, etc.
    for i in range(1, n + 1):
        vertexset.append((n+ k + i - 1, strands[2 * i - 2]))
        vertexset.append((n + k + i - 1, strands[2 * i - 1]))

    # Lastly, we'll make our vertex set go in the negative y direction, it'll make our pictures look how we want them to.
    vsx = [i for i, j in vertexset]
    vsy = [j for i, j in vertexset]
    negvsy = [-x for x in vsy]
    vertexset = list(zip(vsx, negvsy))

    # Here we can initialize where our red edges are, indicating only what x values they're at. This can be changed depending on our needs.
    rededges = list(range(1, n + 1)) + list(range(n + k , (2 * n) + k ))


    #print(vertexset)
    #print(rededges)   #You can uncomment these if you want! 
    
    return [vertexset,rededges]

##############################################################################################
def drawrectdiag(vertexset, rededges):   
#### This will save the tikzfile to your computer as a .txt file
    import numpy as np
    import sys
    sys.stdout = open('RectangularDiagramtizkfile.txt','wt')
    print("\\begin{tikzpicture}")
    # First, we draw the horizontal lines remembering that our y's are negative
    vsy = [j for i, j in vertexset]
    miny: int = min(vsy)
    vsyarray = np.array(vsy)
    for i in range(miny, 0):
        currentvertices = np.where(vsyarray == i)[0]
        vertex1 = vertexset[currentvertices[0]]
        vertex2 = vertexset[currentvertices[1]]
        print("\\draw[black, very thick] ", vertex1, ' -- ', vertex2, ';')

    print('')
    print('')

    # Next, we need to draw the white rectangles because we don't want to figure out how to use the knots package in tikz.
    vsx = [i for i, j in vertexset]
    maxx: int = max(vsx)
    vsxarray = np.array(vsx)
    for i in range(1, maxx + 1):
        currentvertices = np.where(vsxarray == i)[0]
        vertex1 = vertexset[currentvertices[0]]
        vertex2 = vertexset[currentvertices[1]]
        realv1 = (vertex1[0] - .05, vertex1[1] - .25)
        realv2 = (vertex2[0] + .05, vertex2[1] + .25)
        print("\\filldraw[white, very thick] ", realv1, ' rectangle ', realv2, ';')

    print('')
    print('')

    # Next, we draw a small circle around each vertex, and draw red ones on red edges
    for i in vertexset:
        if i[0] in rededges:
            print("\\filldraw[red] ", i, ' circle (1.5pt) node[anchor=west]{};')
        else:
            print("\\filldraw[black] ", i, ' circle (1.5pt) node[anchor=west]{};')

    print('')
    print('')

    # Lastly, we draw the vertical lines, where we need to check for redness.
    for i in range(1, maxx + 1):
        currentvertices = np.where(vsxarray == i)[0]
        vertex1 = vertexset[currentvertices[0]]
        vertex2 = vertexset[currentvertices[1]]
        if i in rededges:
            print("\\draw[red, very thick] ", vertex1, ' -- ', vertex2, ';')
        else:
            print("\\draw[black, very thick] ", vertex1, ' -- ', vertex2, ';')

    print('\\end{tikzpicture}')
#########################################################################################

In [3]:
#This is a prototype of the full function to take a braid word to a rectangular diagram
def braid_word_to_rectangular_diagram(W):
    #The input is an array containing the braid word, for instance sigma_1 sigma_3 sigma_2^{-1} would be input as [1,3,-2].
    
    #Initialization
    n_from_braidgroup=max(abs(i)+1 for i in W) #This is needed for left_normal_to_string function, we need to know the braid group.
    
    #Step 1: Make the braid word all positive
    positiveW=braid_to_positive_braid(W)
    #Step 2: Put it into left-greedy normal form
    W_normal_form=positiveW.left_normal_form()
    #Step 3: Remove the Garside element, turn into a string in a more readable form
    W_normal_form_string=left_normal_to_string(W_normal_form,n_from_braidgroup)
    # Step 4: Partition into increasing substrings.  These are the syllables we need. 
    W_normal_form_syllables=syllable_string_partitioning(W_normal_form_string)
    # Step 5: Break into beginning and end of syllables for plat to rectangular diagram input
    W_input=[[W_normal_form_syllables[i][0],W_normal_form_syllables[i][-1]] for i in range(len(W_normal_form_syllables))]
    
    #Step 6: Input into PlatToRectangularDiagram
    [V,E]=plat2rectdiag(W_input)   #The output printed is the vertex set and the red edges
    drawrectdiag(V, E)  #This will give a tikzpicture for the rectangular diagram! 

    

In [4]:
B4=BraidGroup(4,'s');
B6=BraidGroup(6);
B8=BraidGroup(8);
#########################################################
#Some Hard Unknot Diagrams in plat position
GoeritzUnknot=B4([-2,-2,1,1,-3,-3,2,2,2]);
GoeritzVariationUnknot=B4([-2,-2,-2,1,1,-3,-3,2,2,2,2]);
MortonUnknot = B8([-4,-4,2,1,6,7,3,-4,5,4,4,4,-3,4,1,6,2]);
SolankiUnknot=B4([2,2,-1,2,3,-1,-2,3,-1,-2,-3,-3,-1, 2]);
OchiaiUnknot=B6([-2,4,3,2,4,1,1,5,5,2,-3,2,-3,-4]);

In [7]:
braid_word_to_rectangular_diagram([-2,4,3,2,4,1,1,5,5,2,-3,2,-3,-4])